In [12]:
import numpy as np
import pandas as pd
import math
import sys
from numpy.polynomial import polynomial as P

# ==============================================================================
# CÁC HÀM CÔNG CỤ
# ==============================================================================

def format_polynomial(coeffs, var_name='x'):
    """Tạo chuỗi biểu diễn đa thức từ mảng hệ số [a₀, a₁, a₂, ...]."""
    if np.all(np.abs(coeffs) < 1e-12): return "0.0"
    
    poly_str = []
    if abs(coeffs[0]) > 1e-12:
        poly_str.append(f"{coeffs[0]:.8f}")
        
    for i in range(1, len(coeffs)):
        if abs(coeffs[i]) > 1e-12:
            sign = " + " if coeffs[i] > 0 else " - "
            coeff_val = abs(coeffs[i])
            
            if i == 1:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}")
            else:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}^{i}")
    
    result = "".join(poly_str).strip()
    if result.startswith("+ "):
        result = result[2:]
    return result

def nhandathuc(b_coeffs, c):
    """Nhân đa thức có hệ số b_coeffs (bậc tăng dần) với đơn thức (x - c)."""
    n = len(b_coeffs)
    a_coeffs = np.zeros(n + 1)
    if n > 0:
        a_coeffs[0] = -c * b_coeffs[0]
        for i in range(1, n):
            a_coeffs[i] = b_coeffs[i-1] - c * b_coeffs[i]
        a_coeffs[n] = b_coeffs[n-1]
    return a_coeffs

# ==============================================================================
# CÁC HÀM XỬ LÝ DỮ LIỆU ĐẦU VÀO
# ==============================================================================

def read_data_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        if 'x' not in df.columns or 'y' not in df.columns:
            return None, "LỖI: File phải chứa 2 cột 'x' và 'y'."
        return df, f"✔ Đã đọc thành công {len(df)} dòng dữ liệu."
    except FileNotFoundError:
        return None, f"LỖI: Không tìm thấy file '{file_path}'."
    except Exception as e:
        return None, f"LỖI: Không thể đọc file. Chi tiết: {e}"

def handle_duplicate_x(df):
    duplicates = df[df.duplicated(subset=['x'], keep=False)]
    if not duplicates.empty:
        print("\nCẢNH BÁO: Phát hiện các giá trị x bị trùng lặp.")
        print(duplicates.to_string())
        indices_to_drop = set(duplicates.index)
        while True:
            try:
                keep_index = int(input("Vui lòng nhập chỉ số (index) của dòng muốn GIỮ LẠI: "))
                if keep_index in indices_to_drop:
                    indices_to_drop.remove(keep_index)
                    print(f"✔ Đã giữ lại dòng index {keep_index}.")
                    return df.drop(list(indices_to_drop))
            except (ValueError, KeyError):
                print("LỖI: Vui lòng nhập một số nguyên hợp lệ trong danh sách trên.")
    return df

def check_if_equally_spaced(x_data, tolerance=1e-9):
    if len(x_data) < 2: return False, None
    diffs = np.diff(x_data)
    return np.all(np.abs(diffs - diffs[0]) < tolerance), diffs[0]

# ==============================================================================
# MODULE NỘI SUY NEWTON CHO MỐC BẤT KỲ (TỔNG QUÁT)
# ==============================================================================

def run_newton_arbitrary_nodes(x_data, y_data, method_choice):
    n = len(x_data)
    
    print("\n\n=======================================================================")
    print("B1: XÁC NHẬN CÁC MỐC NỘI SUY")
    print("=======================================================================")
    print(f"-> Sử dụng {n} mốc nội suy đã cho:")
    print(pd.DataFrame({'x': x_data, 'y': y_data}).to_string())

    print("\n\n=======================================================================")
    print("B2: BẢNG TỶ SAI PHÂN VÀ CÁC VECTOR HỆ SỐ")
    print("=======================================================================")
    dd_table = np.zeros((n, n))
    dd_table[:, 0] = y_data
    for j in range(1, n):
        for i in range(j, n):
            dd_table[i, j] = (dd_table[i, j-1] - dd_table[i-1, j-1]) / (x_data[i] - x_data[i-j])
    
    df_display = pd.DataFrame(dd_table); df_display.insert(0, 'X', x_data)
    df_display.columns = ['X'] + ['Y'] + [f'TSP{i}' for i in range(1, n)]
    print(df_display.to_string(index=False))
    
    forward_coeffs = np.diag(dd_table)
    backward_coeffs = dd_table[n-1, :]
    print("\n-> Vector hệ số C_tiến (đường chéo chính):"); print(forward_coeffs)
    print("\n-> Vector hệ số C_lùi (hàng cuối cùng):"); print(backward_coeffs)

    print("\n\n=======================================================================")
    print("B3: CÁC BẢNG TÍCH A (TIẾN VÀ LÙI)")
    print("=======================================================================")
    print("-> Bảng Tích Tiến A_tiến:")
    forward_basis_table = np.zeros((n, n))
    current_poly = np.array([1.0])
    for i in range(n):
        padded_poly = np.pad(current_poly, (0, n - len(current_poly))); forward_basis_table[i, :] = padded_poly
        if i < n - 1: current_poly = nhandathuc(current_poly, x_data[i])
    df_forward_basis_display = pd.DataFrame(np.flip(forward_basis_table, axis=1))
    print(df_forward_basis_display.to_string(header=False, index=False))
    
    print("\n-> Bảng Tích Lùi A_lùi:")
    backward_basis_table = np.zeros((n, n))
    current_poly = np.array([1.0])
    for i in range(n):
        padded_poly = np.pad(current_poly, (0, n - len(current_poly))); backward_basis_table[i, :] = padded_poly
        if i < n - 1: current_poly = nhandathuc(current_poly, x_data[n-1-i])
    df_backward_basis_display = pd.DataFrame(np.flip(backward_basis_table, axis=1))
    print(df_backward_basis_display.to_string(header=False, index=False))

    print("\n\n=======================================================================")
    print("B4: TÍNH HỆ SỐ P(x), ĐA THỨC VÀ KẾT LUẬN")
    print("=======================================================================")
    if method_choice == '1': # Tiến
        standard_coeffs = forward_coeffs @ forward_basis_table
        print("-> Tính hệ số P của P(x) theo công thức P = C_tiến * A_tiến:")
        print("   Vector C_tiến:"); print(f"   {forward_coeffs}")
        print("\n   Ma trận A_tiến:"); print(df_forward_basis_display.to_string(header=False, index=False).replace("\n", "\n   "))
    else: # Lùi
        standard_coeffs = backward_coeffs @ backward_basis_table
        print("-> Tính hệ số P của P(x) theo công thức P = C_lùi * A_lùi:")
        print("   Vector C_lùi:"); print(f"   {backward_coeffs}")
        print("\n   Ma trận A_lùi:"); print(df_backward_basis_display.to_string(header=False, index=False).replace("\n", "\n   "))

    print("\n---> Vector hệ số P (của P(x), bậc tăng dần):"); print(f"   {standard_coeffs}")
    print("\n-> Đa thức nội suy P(x) (duy nhất cho cả 2 cách):")
    print(f"   P(x) = {format_polynomial(standard_coeffs, 'x')}")

    print("\n--- Tính toán giá trị và đạo hàm tại x ---")
    while True:
        try:
            x_target = float(input("Nhập giá trị x cần tính toán: ")); break
        except ValueError: print("Vui lòng nhập một số thực.")

    while True:
        try:
            deriv_order = int(input("Nhập cấp đạo hàm n (nhập 0 để tính P(x)): ")); 
            if deriv_order >= 0: break
        except ValueError: print("Vui lòng nhập một số nguyên không âm.")

    coeffs_for_numpy = np.flip(standard_coeffs)
    result = 0.0
    if deriv_order == 0:
        result = np.polyval(coeffs_for_numpy, x_target)
    elif deriv_order < n:
        poly_deriv = np.polyder(coeffs_for_numpy, deriv_order); result = np.polyval(poly_deriv, x_target)
    
    print(f"-> P^({deriv_order})(x={x_target}) = {result:.8f}")

# ==============================================================================
# MODULE NỘI SUY NEWTON CHO MỐC CÁCH ĐỀU
# ==============================================================================

def run_newton_equally_spaced_nodes(x_full, y_full, h, method_choice):
    while True:
        try:
            num_points = int(input(f"\nNhập số điểm nội suy cần dùng (ví dụ: 8): "));
            if 2 <= num_points <= len(x_full): break
        except ValueError: print("Vui lòng nhập một số nguyên.")
            
    while True:
        try:
            x_target = float(input("Nhập giá trị x cần tính toán (x_bar): ")); break
        except ValueError: print("Vui lòng nhập một số thực.")

    print("\n\n=======================================================================")
    print("B1: TRÍCH XUẤT MỐC")
    print("=======================================================================")
    closest_index = np.argmin(np.abs(x_full - x_target))
    start_index = max(0, min(closest_index - (num_points // 2), len(x_full) - num_points))
    
    x_sub, y_sub = x_full[start_index : start_index + num_points], y_full[start_index : start_index + num_points]
    x0_sub, xn_sub = x_sub[0], x_sub[-1]
    
    print(f"-> Chọn {num_points} mốc [x_{start_index}, ..., x_{start_index + num_points - 1}]")
    print(pd.DataFrame({'x': x_sub, 'y': y_sub}, index=np.arange(start_index, start_index + num_points)).to_string())

    standard_coeffs = [] # Biến lưu hệ số cuối cùng
    if method_choice == '1': # ------ TIẾN ------
        t_target = (x_target - x0_sub) / h
        print(f"\n-> Tính giá trị t_bar = (x_bar - x₀) / h = ({x_target} - {x0_sub}) / {h:.8f} = {t_target:.8f}")

        print("\n\n=======================================================================")
        print("B2: LẬP BẢNG SAI PHÂN TIẾN (Δ)")
        print("=======================================================================")
        f_diff = np.zeros((num_points, num_points)); f_diff[:, 0] = y_sub
        for j in range(1, num_points):
            for i in range(num_points - j): f_diff[i, j] = f_diff[i+1, j-1] - f_diff[i, j-1]
        
        df_display = pd.DataFrame(index=np.arange(start_index, start_index + num_points)); df_display['x'] = x_sub; df_display['y'] = y_sub
        for j in range(1, num_points):
            col_name = f'Δ^{j}y'; col_data = np.full(num_points, np.nan)
            valid_data = f_diff[0 : num_points - j, j]; col_data[j:] = valid_data
            df_display[col_name] = col_data
        print(df_display.to_string())

        coeffs_D = f_diff[0, :]; coeffs_Dy = np.array([f_diff[0, k] / math.factorial(k) for k in range(num_points)])
        print(f"\n-> Vector sai phân tiến (Δᵏy_{start_index}):"); print(coeffs_D)
        print(f"\n-> Vector hệ số Dy (Δᵏy_{start_index} / k!):"); print(coeffs_Dy)
        
        print("\n\n=======================================================================")
        print("B3: LẬP BẢNG TÍCH A CHO P(t)")
        print("=======================================================================")
        prod_table = np.zeros((num_points, num_points)); current_poly = np.array([1])
        for k in range(num_points):
            p = np.flip(current_poly); padded = np.pad(p, (0, num_points - len(p))); prod_table[k, :] = padded
            current_poly = np.polymul(current_poly, [1, -k])
        df_prod_table = pd.DataFrame(np.flip(prod_table, axis=1), dtype=int)
        print(df_prod_table.to_string(header=False, index=False))
        
        print("\n\n=======================================================================")
        print("B4: TÍNH HỆ SỐ P(t), ĐA THỨC VÀ KẾT LUẬN")
        print("=======================================================================")
        coeffs_poly_t = coeffs_Dy @ prod_table
        print("-> Tính hệ số của P(t) theo công thức P = Dy * A:"); print(f"   P = {coeffs_poly_t}")
        print("\n-> Đa thức nội suy P(t):"); print(f"   P(t) = {format_polynomial(coeffs_poly_t, 't')}")
        
        poly_t = P.Polynomial(coeffs_poly_t); poly_x = poly_t(P.Polynomial([-x0_sub/h, 1/h])); standard_coeffs = poly_x.coef
        print("\n-> Đa thức nội suy P(x) dạng chính tắc:"); print(f"   P(x) = {format_polynomial(standard_coeffs, 'x')}")
        
    else: # ------ LÙI ------
        s_target = (x_target - xn_sub) / h
        print(f"\n-> Tính giá trị s_bar = (x_bar - xₙ) / h = ({x_target} - {xn_sub}) / {h:.8f} = {s_target:.8f}")
        
        print("\n\n=======================================================================")
        print("B2: LẬP BẢNG SAI PHÂN LÙI (∇)")
        print("=======================================================================")
        b_diff = np.zeros((num_points, num_points)); b_diff[:, 0] = y_sub
        for j in range(1, num_points):
            for i in range(j, num_points): b_diff[i, j] = b_diff[i, j-1] - b_diff[i-1, j-1]
        
        df_display = pd.DataFrame(b_diff, index=x_sub); df_display.columns = ['y'] + [f'∇^{i}y' for i in range(1, num_points)]
        print(df_display.to_string())

        coeffs_D = b_diff[-1, :]; coeffs_Ds = np.array([b_diff[-1, k] / math.factorial(k) for k in range(num_points)])
        print(f"\n-> Vector sai phân lùi (∇ᵏy_{start_index+num_points-1}):"); print(coeffs_D)
        print(f"\n-> Vector hệ số Ds (∇ᵏyₙ / k!):"); print(coeffs_Ds)
        
        print("\n\n=======================================================================")
        print("B3: LẬP BẢNG TÍCH A CHO P(s)")
        print("=======================================================================")
        prod_table = np.zeros((num_points, num_points)); current_poly = np.array([1])
        for k in range(num_points):
            p = np.flip(current_poly); padded = np.pad(p, (0, num_points - len(p))); prod_table[k, :] = padded
            current_poly = np.polymul(current_poly, [1, k]) # s(s+1)...
        df_prod_table = pd.DataFrame(np.flip(prod_table, axis=1), dtype=int)
        print(df_prod_table.to_string(header=False, index=False))
        
        print("\n\n=======================================================================")
        print("B4: TÍNH HỆ SỐ P(s), ĐA THỨC VÀ KẾT LUẬN")
        print("=======================================================================")
        coeffs_poly_s = coeffs_Ds @ prod_table
        print("-> Tính hệ số của P(s) theo công thức P = Ds * A:"); print(f"   P = {coeffs_poly_s}")
        print("\n-> Đa thức nội suy P(s):"); print(f"   P(s) = {format_polynomial(coeffs_poly_s, 's')}")
        
        poly_s = P.Polynomial(coeffs_poly_s); poly_x = poly_s(P.Polynomial([-xn_sub/h, 1/h])); standard_coeffs = poly_x.coef
        print("\n-> Đa thức nội suy P(x) dạng chính tắc:"); print(f"   P(x) = {format_polynomial(standard_coeffs, 'x')}")

    print("\n--- Tính toán giá trị và đạo hàm tại x_bar ---")
    while True:
        try:
            deriv_order = int(input("Nhập cấp đạo hàm n (nhập 0 để tính P(x)): ")); 
            if deriv_order >= 0: break
        except ValueError: print("Vui lòng nhập một số nguyên không âm.")

    coeffs_for_numpy = np.flip(standard_coeffs)
    result = 0.0
    if deriv_order == 0:
        result = np.polyval(coeffs_for_numpy, x_target)
    elif deriv_order < num_points:
        poly_deriv = np.polyder(coeffs_for_numpy, deriv_order)
        result = np.polyval(poly_deriv, x_target)
    
    print(f"-> P^({deriv_order})(x={x_target}) = {result:.8f}")
    
# ==============================================================================
# HÀM CHÍNH ĐIỀU PHỐI CHƯƠNG TRÌNH
# ==============================================================================

def main():
    np.set_printoptions(suppress=True, precision=8)
    pd.options.display.float_format = '{:.8f}'.format

    print("=======================================================================")
    print("CHƯƠNG TRÌNH NỘI SUY NEWTON TỔNG HỢP")
    print("=======================================================================")
    file_path = "data/input_newton.xlsx"

    df, msg = read_data_from_excel(file_path); print(msg)
    if df is None: sys.exit()

    df_cleaned = handle_duplicate_x(df)
    x_data, y_data = df_cleaned['x'].to_numpy(), df_cleaned['y'].to_numpy()
    is_equally, h = check_if_equally_spaced(x_data)

    if is_equally:
        print(f"\n[PHÂN TÍCH] Dữ liệu có các mốc CÁCH ĐỀU, h ≈ {h:.6f}")
        while True:
            choice = input("\nBạn muốn chọn phương pháp nào?\n1. Nội suy cho Mốc CÁCH ĐỀU (khuyến khích).\n2. Dùng phương pháp TỔNG QUÁT cho Mốc Bất Kỳ.\nLựa chọn (1 hoặc 2): ")
            if choice in ['1', '2']: break
        
        if choice == '1':
            while True:
                method_choice = input("\nChọn thuật toán:\n1. Sai phân TIẾN.\n2. Sai phân LÙI.\nLựa chọn (1 hoặc 2): ")
                if method_choice in ['1', '2']: break
            run_newton_equally_spaced_nodes(x_data, y_data, h, method_choice)
        else:
            while True:
                method_choice = input("\nChọn thuật toán:\n1. Newton TIẾN.\n2. Newton LÙI.\nLựa chọn (1 hoặc 2): ")
                if method_choice in ['1', '2']: break
            run_newton_arbitrary_nodes(x_data, y_data, method_choice)
    else:
        print("\n[PHÂN TÍCH] Dữ liệu có các mốc KHÔNG CÁCH ĐỀU.")
        print("--> Tự động sử dụng phương pháp Nội suy Newton Tổng Quát.")
        while True:
            method_choice = input("\nChọn thuật toán:\n1. Newton TIẾN.\n2. Newton LÙI.\nLựa chọn (1 hoặc 2): ")
            if method_choice in ['1', '2']: break
        run_newton_arbitrary_nodes(x_data, y_data, method_choice)

if __name__ == "__main__":
    main()

CHƯƠG TRÌNH NỘI SUY NEWTON TỔNG HỢP
✔ Đã đọc thành công 201 dòng dữ liệu.

[PHÂN TÍCH] Dữ liệu có các mốc CÁCH ĐỀU, h ≈ 0.118000



Bạn muốn chọn phương pháp nào?
1. Nội suy cho Mốc CÁCH ĐỀU (khuyến khích).
2. Dùng phương pháp TỔNG QUÁT cho Mốc Bất Kỳ.
Lựa chọn (1 hoặc 2):  1

Chọn thuật toán:
1. Sai phân TIẾN.
2. Sai phân LÙI.
Lựa chọn (1 hoặc 2):  2

Nhập số điểm nội suy cần dùng (ví dụ: 8):  5
Nhập giá trị x cần tính toán (x_bar):  4




B1: TRÍCH XUẤT MỐC
-> Chọn 5 mốc [x_23, ..., x_27]
            x          y
23 3.71400000 2.82670000
24 3.83200000 2.43840000
25 3.95000000 2.03540000
26 4.06800000 1.62340000
27 4.18600000 1.20840000

-> Tính giá trị s_bar = (x_bar - xₙ) / h = (4.0 - 4.185999999999997) / 0.11800000 = -1.57627119


B2: LẬP BẢNG SAI PHÂN LÙI (∇)
           0           1           2          3          4
0 2.82670000  0.00000000  0.00000000 0.00000000 0.00000000
1 2.43840000 -0.38830000  0.00000000 0.00000000 0.00000000
2 2.03540000 -0.40300000 -0.01470000 0.00000000 0.00000000
3 1.62340000 -0.41200000 -0.00900000 0.00570000 0.00000000
4 1.20840000 -0.41500000 -0.00300000 0.00600000 0.00030000

-> Vector sai phân lùi (∇ᵏy_27):
[ 1.2084 -0.415  -0.003   0.006   0.0003]

-> Vector hệ số Ds (∇ᵏyₙ / k!):
[ 1.2084    -0.415     -0.0015     0.001      0.0000125]


B3: LẬP BẢNG TÍCH A CHO P(s)
0 0  0 0 1
0 0  0 1 0
0 0  1 1 0
0 1  3 2 0
1 6 11 6 0


B4: TÍNH HỆ SỐ P(s), ĐA THỨC VÀ KẾT LUẬN
-> Tính hệ số của P